# Fine-tuning a model with the Trainer API
# 使用Trainer微調模型

 Transformers 提供了一個`Trainer`類別來**微調**資料集上提供的任何預訓練模型。完成資料預處理工作後，只需執行幾個步驟即可定義Trainer, 準備運行環境`Trainer.train()`

安裝套件

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 26.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-m

In [ ]:
# 從 Hugging Face 的 datasets 庫中加載 GLUE 任務中的 MRPC（Microsoft Research Paraphrase Corpus）數據集
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

# 加載 GLUE 的 MRPC 任務數據集
raw_datasets = load_dataset("glue", "mrpc")

# 指定要使用的模型檢查點（這裡使用的是 BERT 基礎未經過大小寫區分的版本）
checkpoint = "bert-base-uncased"

# 從 Hugging Face Hub 加載對應於指定檢查點的分詞器
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# 定義一個函數來對數據集中的句子對進行分詞
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

# 將分詞函數應用於數據集中的每一個示例，並啟用批處理
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

# 創建一個 DataCollatorWithPadding 對象，用於在批處理時自動對序列進行填充
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

### 定義`Trainer`的參數
我們定義我們之前的第一步`Trainer`是定義一個類別，其中包含將用於訓練和評估的`TrainingArguments`所有超參數。為求簡單，這邊`Trainer`提供的參數是儲存訓練模型的目錄以及訓練的輪數。對於其餘所有內容，保留預設值，這對於基本的微調應該非常有效。

In [ ]:
# 從 transformers 庫中導入 TrainingArguments 類
from transformers import TrainingArguments

# 創建一個 TrainingArguments 對象，用於設置訓練過程中的參數
# "test-trainer" 是訓練過程中輸出文件保存的目錄名
# num_train_epochs=10 指定訓練的總輪數為 10
training_args = TrainingArguments("test-trainer", num_train_epochs=10)


使用`AutoModelForSequenceClassification`有兩個標籤的類別

In [ ]:
# 從 transformers 庫中導入 AutoModelForSequenceClassification 類
from transformers import AutoModelForSequenceClassification

# 使用指定的檢查點（checkpoint）加載一個預訓練的序列分類模型
# num_labels=2 指定模型的輸出層具有 2 個標籤，適用於二分類任務
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 說明
在實例化這個預訓練模型後會收到一個警告。這是因為BERT沒有在對句子對進行分類的任務上進行預訓練，因此預訓練模型的頭部被丟棄了(bert-base-uncase)，並且加入了一個適合序列分類的新頭部。表示有些權重未被使用（對應於被丟棄的預訓練頭部的那些）以及一些其他權重是隨機初始化的（新頭部）

### 定義`Trainer`
有了模型，就可以傳入建構的所有對象來定義一個訓練器模型、`training_args`、訓練和驗證資料集、`data_collator`和分詞器

In [ ]:
# 從 transformers 庫中導入 Trainer 類
from transformers import Trainer

# 創建一個 Trainer 對象，用於管理訓練和評估過程
trainer = Trainer(
    model,  # 指定要訓練的模型
    training_args,  # 指定訓練參數
    train_dataset=tokenized_datasets["train"],  # 訓練數據集
    eval_dataset=tokenized_datasets["validation"],  # 驗證數據集
    data_collator=data_collator,  # 用於在批處理時自動對序列進行填充的數據填充器
    tokenizer=tokenizer,  # 分詞器，用於將文本轉換為模型可以處理的格式
)


### 說明
這樣傳遞分詞器時，訓練器使用的預設`data_collator`將是之前定義的`DataCollatorWithPadding`，因此可以跳過`data_collator=data_collator`這一行

### 開始訓練
呼叫`Trainer`中的`train()`方法

In [ ]:
trainer.train()

Step,Training Loss
500,0.587200
1000,0.436000
1500,0.311000
2000,0.194500
2500,0.109400
3000,0.057300
3500,0.038400
4000,0.028800
4500,0.021600


TrainOutput(global_step=4590, training_loss=0.19458388349848896, metrics={'train_runtime': 713.5929, 'train_samples_per_second': 51.402, 'train_steps_per_second': 6.432, 'total_flos': 1353042435523440.0, 'train_loss': 0.19458388349848896, 'epoch': 10.0})

### 計算模型表現的度量方法
- 計算408個預測中，有幾個是和真值一模一樣的

In [ ]:
# 導入 evaluate 庫和 numpy 庫
import evaluate
import numpy as np

# 使用訓練好的模型對驗證數據集進行預測
predictions = trainer.predict(tokenized_datasets["validation"])

# 從預測結果中選擇得分最高的標籤作為最終預測
preds = np.argmax(predictions.predictions, axis=-1)

# 再次使用訓練好的模型對驗證數據集進行預測
predictions = trainer.predict(tokenized_datasets["validation"])

# 加載 GLUE 評測指標，用於 MRPC 任務
metric = evaluate.load("glue", "mrpc")

# 計算預測結果和真實標籤之間


{'accuracy': 0.8382352941176471, 'f1': 0.8888888888888888}